In [26]:
from MetaSet import advancedStructure as ad
import numpy as np

def main(meta, name):
    meta.materialSet()
    meta.baseBuild(0.4e-6)
    meta.structureBuild(2, [0.1e-6], 0.6e-6)
    meta.fdtd.save(name)
    meta.fdtd.addjob(name)
    meta.fdtd.close()

meta=ad.MetaEngine()
meta.materialSet()
meta.baseBuild(0.4e-6)
meta.structureBuild(4, [0.4e-6,0.1e-6,0.12e-6], 0.6e-6)

for i in range(4):
    name=f'test{i}.fsp'
    meta.fdtd.save(name)
    meta.fdtd.addjob(name)

In [27]:
meta.fdtd.runjobs()

In [28]:
for i in range(4):
    name=f'test{i}.fsp'
    meta.fdtd.load(name)
    Trans = meta.fdtd.transmission("plane")
    print(Trans)

[[0.95894783]
 [0.72284618]
 [0.97927418]
 [0.08281982]
 [0.36269824]]
[[0.95894783]
 [0.72284618]
 [0.97927418]
 [0.08281982]
 [0.36269824]]
[[0.95894783]
 [0.72284618]
 [0.97927418]
 [0.08281982]
 [0.36269824]]
[[0.95894783]
 [0.72284618]
 [0.97927418]
 [0.08281982]
 [0.36269824]]


In [8]:
meta.fdtd.load('test0.fsp')
Trans = meta.fdtd.transmission("plane")
print(Trans)

[[0.97893937]
 [0.96628934]
 [0.96716401]
 [0.9762918 ]
 [0.94940644]]
